# Lab Assignment Three: Extending Logistic Regression

### Authors
- Juliana Antonio
- Xiaona Hang
- Chuanqi Deng


### 1. Preparation and Overview

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression as SKLogisticRegression
from numpy.linalg import pinv
from scipy.special import expit
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from tqdm import tqdm

raw_data = pd.read_csv("data/bodyPerformance.csv")

# preprocessing
data = pd.get_dummies(raw_data, columns=['gender'],dtype=np.int8) # one-hot encoding for gender
labels = data['class'].map(lambda c: ord(c) - ord('A')) # encode lables into integer
data.drop(['class'], axis=1, inplace=True) # remove class column

# scale
scaler = StandardScaler()
features = scaler.fit_transform(data)

# PCA
pca = PCA(8)
pca_feature = pca.fit_transform(features)

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pca_feature, labels, train_size=0.8)
print(pca_feature.shape, X_train.shape, X_test.shape,  y_train.shape, y_test.shape)

(13393, 8) (10714, 8) (2679, 8) (10714,) (2679,)


In [2]:
data.describe()

,age,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,gender_F,gender_M
count,13393.000000,13393.000000,13393.000000,13393.000000,13393.000000,13393.000000,13393.000000,13393.000000,13393.000000,13393.000000,13393.000000,13393.000000
mean,36.775106,168.559807,67.447316,23.240165,78.796842,130.234817,36.963877,15.209268,39.771224,190.129627,0.367804,0.632196
std,13.625639,8.426583,11.949666,7.256844,10.742033,14.713954,10.624864,8.456677,14.276698,39.868000,0.482226,0.482226
min,21.000000,125.000000,26.300000,3.000000,0.000000,0.000000,0.000000,-25.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,162.400000,58.200000,18.000000,71.000000,120.000000,27.500000,10.900000,30.000000,162.000000,0.000000,0.000000
50%,32.000000,169.200000,67.400000,22.800000,79.000000,130.000000,37.900000,16.200000,41.000000,193.000000,0.000000,1.000000
75%,48.000000,174.800000,75.300000,28.000000,86.000000,141.000000,45.200000,20.700000,50.000000,221.000000,1.000000,1.000000
max,64.000000,193.800000,138.100000,78.400000,156.200000,201.000000,70.500000,213.000000,80.000000,303.000000,1.000000,1.000000


#### Performance Comparison with Scikit-learn

In [3]:
%%time
lr_sk = SKLogisticRegression(solver='liblinear') # all params default
lr_sk.fit(X_train,y_train)
print(np.hstack((lr_sk.intercept_[:,np.newaxis],lr_sk.coef_)))
yhat = lr_sk.predict(X_test)

accuracy = accuracy_score(y_test, yhat)
#precision = precision_score(y_test, yhat, average='weighted')  
#recall = recall_score(y_test, yhat, average='weighted')  
#f1 = f1_score(y_test, yhat, average='weighted')  

print("Accuracy:", accuracy)
#print("Precision:", precision)
#print("Recall:", recall)
#print("F1 Score:", f1)

[[-2.41977467 -0.09545565  1.11821723  1.86985903 -1.01044404  1.09803135
   0.59163907 -0.07202187 -1.58050737]
 [-1.16996009 -0.03804189  0.16283534  0.3216058  -0.28314733  0.18604046
   0.19843615 -0.03550531 -0.19669247]
 [-1.12259746 -0.01149002 -0.0079301  -0.11891965 -0.12635462 -0.2278588
  -0.33620969 -0.01834717  0.32161574]
 [-2.1575926   0.1626009  -1.25220351 -1.7772739   1.36972476 -0.6240763
  -0.27301052  0.19085972  1.53699438]]
Accuracy: 0.5808137364688316
CPU times: user 58.1 ms, sys: 3.86 ms, total: 62 ms
Wall time: 60.3 ms


### 3. Deployment

Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party implementation)? Why?


### 4. Exceptional Work

Implement an optimization technique for logistic regression using mean square error as your objective function (instead of maximum likelihood). Derive the gradient updates for the Hessian and use Newton's method to update the values of "w". Then answer, which process do you prefer: maximum likelihood OR minimum mean-squared error?

#### Mean Squared Error (MSE) Approach 

In logistic regression, the Mean Squared Error (MSE) approach minimizes the squared difference between predicted probabilities and actual labels.

$$
\text{MSE} = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2
$$

The gradient of the MSE loss with respect to the weights is computed as:

$$
\nabla l(\mathbf{w}) = \frac{1}{N} \mathbf{X}^T (\mathbf{y} - \hat{\mathbf{y}})
$$

where:
- $\mathbf{X}$ is the design matrix
- $\mathbf{y}$ is the true labels
- $\hat{\mathbf{y}}$ is the predicted labels

We compute the gradient of the MSE loss function with respect to the model parameters and update the parameters iteratively until convergence.

The Hessian matrix is computed as:

$$
\mathbf{H} = \frac{1}{N} \mathbf{X}^T \mathbf{X}
$$

Using Newton's method, we iteratively update the weights as:

$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - (\mathbf{H}^{-1} \nabla l(\mathbf{w}_{\text{old}}))
$$


In [43]:
class LogisticRegressionMSE:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        self.weights = None  # Initialize weights
        
    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.w_ = np.zeros((num_features + 1, 1))
        X = np.c_[np.ones((num_samples, 1)), X]  # Add intercept
        
        for _ in range(self.iters):
            y_pred = self._sigmoid(X @ self.w_)
            y_pred = y_pred.flatten()
            gradient = X.T @ (y_pred - y) / num_samples
            hessian = X.T @ X / num_samples
            gradient = gradient.reshape(-1, 1) # Reshape gradient to ensure it's treated as a column vector
            self.w_ -= self.eta * np.linalg.inv(hessian) @ gradient # Update the model parameters using Newton's method
        
        self.weights = self.w_  # Store weights
    
    def predict_proba(self, X):
        X = np.c_[np.ones((X.shape[0], 1)), X]
        return self._sigmoid(X @ self.w_)
    
    def predict(self, X):
        probabilities = self.predict_proba(X)
        return (probabilities >= 0.5).astype(int)
    
    def _sigmoid(self, z):
        z_clipped = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z_clipped))

In [44]:
%%time
from sklearn.metrics import accuracy_score

lr_mse = LogisticRegressionMSE(eta=0.1, iterations=1000)

lr_mse.fit(X_train, y_train)
y_pred = lr_mse.predict(X_test)
print(np.vstack((lr_mse.weights[0], lr_mse.weights[1:])))
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


[[ 65.04529757]
 [  2.90775018]
 [-24.89615449]
 [-38.29149947]
 [ 24.1365686 ]
 [-17.57498127]
 [-11.99674579]
 [  3.47479731]
 [ 36.65041944]]
Accuracy: 0.341545352743561
CPU times: user 3.38 s, sys: 184 ms, total: 3.56 s
Wall time: 470 ms


References: